In [2]:
import os
import json
import argparse
import yfinance as yf
import openai

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise EnvironmentError("OPENAI_API_KEY 환경 변수를 설정해주세요.")

In [4]:
def get_stock_data(ticker: str, start: str, end: str) -> str:
    """
    Yahoo Finance에서 주가 데이터를 조회하여 CSV 문자열로 반환합니다.
    ticker: 종목 티커 (예: NVDA, 005930.KS)
    start, end: YYYY-MM-DD 포맷의 시작/종료 날짜
    """
    df = yf.download(ticker, start=start, end=end)
    if df.empty:
        return json.dumps({"error": "No data found for {ticker} between {start} and {end}."}, ensure_ascii=False)
    return df.to_csv()

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_data",
            "description": "지정된 종목과 기간의 주가 데이터를 CSV 형태로 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "주가 조회할 종목 티커"},
                    "start":  {"type": "string", "description": "조회 시작 날짜(YYYY-MM-DD)"},
                    "end":    {"type": "string", "description": "조회 종료 날짜(YYYY-MM-DD)"}
                },
                "required": ["ticker", "start", "end"]
            }
        }
    }
]

In [8]:
parser = argparse.ArgumentParser(description="Stock Analysis Tool via function calls")
parser.add_argument('--ticker', required=True, help='조회할 종목 티커')
parser.add_argument('--start',  required=True, help='조회 시작 날짜 YYYY-MM-DD')
parser.add_argument('--end',    required=True, help='조회 종료 날짜 YYYY-MM-DD')
args = parser.parse_args()

# 4) 초기 대화 메시지 구성
messages = [
    {"role": "system", "content": "당신은 금융 분석 전문 어시스턴트입니다."},
    {"role": "user",   "content": (f"{args.ticker}의 주가를 {args.start}부터 {args.end}까지 분석해서 보고서를 작성해 주세요.")}
]

# 5) function(tool) 호출
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)



usage: ipykernel_launcher.py [-h] --ticker TICKER --start START --end END
ipykernel_launcher.py: error: the following arguments are required: --ticker, --start, --end


SystemExit: 2

In [7]:
%run stock_tool.py --ticker NVDA --start 2025-06-01 --end 2025-07-21

Exception: File `'stock_tool.py'` not found.